In [89]:
import numpy as np
import pywt
import scipy
data = np.load("denoise.npy")

In [90]:
levels = max(pywt.dwt_max_level(data.size, 2) - 3, 1)

In [91]:
pywt.wavedecn(data, wavelet="db38", level=levels)[0]

array([[1.75599837e+08, 1.75599837e+08, 1.75599837e+08, ...,
        1.75599837e+08, 1.75599837e+08, 1.75599837e+08],
       [2.07686683e+08, 2.07686683e+08, 2.07686683e+08, ...,
        2.07686683e+08, 2.07686683e+08, 2.07686683e+08],
       [2.13669606e+08, 2.13669606e+08, 2.13669606e+08, ...,
        2.13669606e+08, 2.13669606e+08, 2.13669606e+08],
       ...,
       [1.35313768e+09, 1.35313768e+09, 1.35313768e+09, ...,
        1.35313768e+09, 1.35313768e+09, 1.35313768e+09],
       [2.13710380e+09, 2.13710380e+09, 2.13710380e+09, ...,
        2.13710380e+09, 2.13710380e+09, 2.13710380e+09],
       [7.82006576e+08, 7.82006576e+08, 7.82006576e+08, ...,
        7.82006576e+08, 7.82006576e+08, 7.82006576e+08]])

In [92]:
pywt.dwt(data, pywt.Wavelet("db1"))[0]

array([[ 1764958.3248315 ],
       [ 1071961.15235674],
       [ -494284.61061215],
       ...,
       [10901401.6059427 ],
       [16118796.06199545],
       [ 6684806.49000164]])

In [93]:
data += np.array(np.random.normal(0, 100000, data.shape), dtype=np.int32)

In [94]:
waveletType = "db38"
original_extent = tuple(slice(s) for s in data.shape)
coeffs = pywt.wavedecn(data, wavelet=waveletType, level=levels)
dcoeffs = coeffs[1:]
detail_coeffs = dcoeffs[-1]['dd']
detail_coeffs = detail_coeffs[np.nonzero(detail_coeffs)]
denom = scipy.stats.norm.ppf(0.75)
sigma = np.median(np.abs(detail_coeffs)) / denom
threshold = sigma * np.sqrt(2 * np.log(data.size))
denoised_detail = [{
    key: pywt.threshold(level[key], value=threshold, mode="soft") for key in level
} for level in dcoeffs]
denoised_coeffs = [coeffs[0]] + denoised_detail
np.sum(np.abs(pywt.waverecn(denoised_coeffs, waveletType)[original_extent] - data))

2.7719856007024646e-05